<b> Diet Problem </b>

There are 5 types of food and 2 nutrient requirements that we must satisfy at minimum cost. We are given the nutritional content and cost per ounce of each food type.

<table>
<caption>Units of nutrients and cost per ounce</caption>
<tr>
<th> Food type </th> <th> Iron </th> <th> Calcium </th> <th> Cost </th>
</tr>
<tr>
<th> 1 </th> <th> 2 </th> <th> 0 </th> <th> 20 </th>
</tr>
<tr>
<th> 2 </th> <th> 0 </th> <th> 1 </th> <th> 10 </th>
</tr>
<tr>
<th> 3 </th> <th> 3 </th> <th> 2 </th> <th> 31 </th>
</tr>
<tr>
<th> 4 </th> <th> 1 </th> <th> 2 </th> <th> 11 </th>
</tr>
<tr>
<th> 5 </th> <th> 2 </th> <th> 1 </th> <th> 12 </th>
</tr>

What is the minimum cost combination of these foods that provides at least 21 units of iron and 12 units of calcium?

<b> Diet Problem Formulation </b>
<ul> 
<li> Decision Variables: 
<ul type="square">
<li>$x_j$ = number of ounces of food type $j=1,\ldots,5$</li>
</ul>
<li> Objective Function: 
<ul type="square">
<li> min $z = 20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5$ </li>
</ul>
</li>
<li> Structural Constraints: </li>
<ul type="square">
<li> $2 x_1 + 0 x_2 + 3 x_3 + 1 x_4 + 2 x_5 \ge 21$ (iron requirement) </li>
<li> $0 x_1 + 1 x_2 + 2 x_3 + 2 x_4 + 1 x_5 \ge 12$ (calcium requirement) </li>
</ul>
<li> Nonnegativity constraints
<ul type="square">
<li> $x_j \ge 0,j=1,\ldots,5$ </li>
</li>
</ul>

<b>Building the Model with gurobipy</b>

We'll first need to import the gurobipy module. Our prefered alias is 'grb'. We'll also create an alias for gurobipy.GRB, which contains some useful constants we will typically need.

In [ ]:
import gurobipy as grb
GRB = grb.GRB

The gurobipy.Model object serves as a repository for all data pertaining to your optimization problem, and provides methods for instantiating decision variables and constraints.

In [ ]:
m = grb.Model()

You can name your gurobipy.Model object anything, but we've chosen "m" here. This object provides methods to instantiate decision variables (addVar) and structural constraints (addConstr).

<b>Adding Variables</b>

We'll add the decision variables first using the Model.addVar() method.

In [ ]:
?m.addVar

This method takes in a variety of optional parameters and returns a reference to a gurobipy.Var object. You will need to save these references in order to build structural constraints later on. If you know lower or upper bounds on the variables, those bounds should be passed in through the lb and ub arguments, which default to 0 and infinity, respectively. Also at this point you have the option to specify an objective coefficient through the obj argument. 

It is optional but generally good practice to provide meaningful names for each of your variables. We will ultimately want to output and view the model we have built to verify that it is what we intended, and the names you have set here will appear in that output. Those names need not coincide with the names of the Python variables used to reference the variables, which means they need not follow rules for naming Python variables. A good naming convention is to use a word or short phrase describing what the variable represents, followed by a period separated list of the variable indices (in this case the food types 1 through 5).

In [ ]:
x1 = m.addVar(lb=0, ub=GRB.INFINITY, obj=20, vtype=GRB.CONTINUOUS, name='consumed.1')

In [ ]:
x2 = m.addVar(obj=10, name='consumed.2')

In [ ]:
x3 = m.addVar(obj=31, name='consumed.3')

In [ ]:
x4 = m.addVar(obj=11, name='consumed.4')

In [ ]:
x5 = m.addVar(obj=12, name='consumed.5')

In [ ]:
# At this point we can print what we have saved from the calls to Model.addVar().
print x1, x2, x3, x4, x5

We see that x1,...,x5 are all Var objects but they are said to be "Awaiting Model Update." Every variable we add expands the size of the model. As a performance enhancement, Gurobi will not officially add variables to the model until you tell the Model object you are done adding variables by calling Model.update().

Note: Gurobi 6.5 provides a new parameter UpdateMode, which when set to 1 will eliminate the need to call Model.update() in most settings. That being said, it is a good practice to add variables in bulk before switching over to adding constraints. Ideally, the number of times you switch from adding variables to adding constraints (or vice versa) should not depend on the size of the problem.

In [ ]:
m.update()
print x1, x2, x3, x4, x5

<b>Building Linear Expressions </b>

Structural constraints involve multiple decision variables. The gurobipy.LinExpr object stores a linear function of decision variables. The + and * operators are overloaded so you can create linear expressions in a natural way.

In [ ]:
lhs = 2*x1 + 3*x3 + x4 + 2*x5
print lhs

<b>Adding Structural Constraints</b>

To add a structural constraint, call the addConstr method on a Model object.

In [ ]:
?m.addConstr

We'll demonstrate two ways to call Model.addConstr() here. We prefer the second option as it has a more explicit connection to the constraint in the model above.

In [ ]:
iron_constr = m.addConstr(lhs, GRB.GREATER_EQUAL, 21, name='nutrient.iron')

The ==, >=, and <= operators have also been overloaded so that constraints can be written in a natural way.

In [ ]:
calcium_constr = m.addConstr(x2 + 2*x3 + 2*x4 + x5 >= 12, name='nutrient.calcium')

As with variables, it is good practice to follow a consistent naming convention with constraints. Those names will help you identify constraints when you eventually view the full model.

When all constraints are created, they can be added to the model in batch by calling the update method.

In [ ]:
m.update()

<b>Inspecting the Model</b>

We can optimize at this point, but it is nice to have some assurance that the code we just wrote actually built the model we hope it built. The .lp file format provides a human-readable representation of the model.

In [ ]:
m.write('diet.lp')

In [ ]:
!more diet.lp

<b>Solving the Model</b>

If that looks good, we are ready to solve the model, which we do by calling Model.optimize(). Afterwords, we can check the model's Status attribute to see if the solve was successful. If the status is equal to GRB.OPTIMAL, then Gurobi was able to find a provably optimal solution (within a tolerance).

In [ ]:
m.optimize()
print "Model status =", m.Status
assert m.Status == GRB.OPTIMAL

The call to Model.optimize() produces a log that gives various model statistics such as the number of variables, constraints, and non-zero entries of the constraint matrix, as well as ranges for all numerical inputs. For longer solves, the log will also include a table which Gurobi periodically writes to in order to keep you informed of the progress. If an optimal solution is found, the log will finish with the optimal objective value.

<b>Querying the Solution</b>

The log provides an optimal objective value but not optimal values of the variables, which are usually the most important output of the solve as they are what ultimately drive your decisions. To obtain solution data, we can query various attributes of the Var and Constr objects that we saved earlier.

We'll look at two of these Var attributes here. The first and typically most important attribute is simply named 'X' and provides the value of the variable in the optimal solution. For the diet problem, this tells us the number of ounces of each food type we should include in our diet. We can use the Var references saved earlier to query this attribute.

In [ ]:
print x1.VarName, x1.X
print x2.VarName, x2.X
print x3.VarName, x3.X
print x4.VarName, x4.X
print x5.VarName, x5.X

The second Var attribute we'll look at here is the reduced cost, which has the attribute name 'RC'. This reduced cost tells us the amount by which a variable's objective coefficient would have to change in order for the variable to have a positive value in an optimal solution. For the variables x.4 and x.5 that are already positive, the reduced cost is always zero.

In [ ]:
print x1.VarName, x1.RC
print x2.VarName, x2.RC
print x3.VarName, x3.RC
print x4.VarName, x4.RC
print x5.VarName, x5.RC

The interpretation here is that food type 1 would have to be at least $11 \frac{1}{3}$ cheaper for it to become cost effective for us to include it in our diet.

You can iterate over all variables in the model with the Model.getVars() method.

In [ ]:
for var in m.getVars():
    print var.VarName, var.X, var.RC

The objective value appears in the solution log but can be queried via the Model attribute ObjVal.

In [ ]:
print m.ObjVal

There is also solution information associated with constraints that can be useful for some applications. Here, we'll look at the Slack and Pi attributes of the Constr object.

In [ ]:
for constr in m.getConstrs():
    print constr.ConstrName, constr.Slack, constr.Pi

The Slack attribute gives the difference between the right-hand and left-hand sides of the constraint in the optimal solution. The Pi attribute is also known as the dual value or shadow price, and tell us how much the objective would change if the right-hand side of the constraint were increased by 1 unit. A constraint may have either a positive slack or a positive dual value, but never both simultateously positive. (Why not?)

Here, neither constraint has slack and both have positive dual values. The interpretation for iron would be that increasing the iron requirement by 1 unit (thereby increasing the right-hand side of the iron constraint by 1 unit) would increase the optimal cost of our diet by $4 \frac{1}{3}$. This is effectively saying that the maximum price we'd be willing to pay for a unit of iron should be $4 \frac{1}{3}$.

Our structural constraints frequently enforce some sort of minimum resource requirement in order to operate a system. When this is the case, the dual variables can be interpreted as a price for those resources.